In [ ]:
import numpy as np
import cv2
import glob
import os
import math
import random

In [ ]:
images = []
ground_truth = []

for image in glob.glob('./raw/*'):
    images.append(image)
    
for gt in glob.glob('./warped/*'):
    ground_truth.append(gt)

In [ ]:
os.mkdir('./flipped/')
os.mkdir('./rotated/')

In [ ]:
os.mkdir('./flipped/raw/')
os.mkdir('./flipped/warped/')
os.mkdir('./rotated/raw/')
os.mkdir('./rotated/warped')

In [ ]:
def flip_horizontal(image):
    image = cv2.imread(image)
    flipHorizontal = cv2.flip(image, 1)
    return flipHorizontal

https://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders

In [ ]:
def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result

In [ ]:
def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )
      

In [ ]:
def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

In [ ]:
def generate_rotated_image(image,angle):
    image = cv2.imread(image)
    image_height, image_width = image.shape[0:2]

    image_rotated = rotate_image(image, angle)
    image_rotated_cropped = crop_around_center(
        image_rotated,
        *largest_rotated_rect(
            image_width,
            image_height,
            math.radians(angle)
        )
    )

    return image_rotated_cropped

The code below flip horizontally the image and rotate the image ramdonly to a positive and negative angle

In [ ]:
angles = [5,8,10]
neg_angles = [-5,-8,-10]

for i in range(len(images)):
    image = images[i].split("\\")[1].split('.')[0]
    gt = ground_truth[i].split("\\")[1].split('.')[0]
    cv2.imwrite(f"./flipped/raw/{image}_flipped.jpg", flip_horizontal(images[i]))
    cv2.imwrite(f"./flipped/warped/{gt}_flipped.jpg", flip_horizontal(ground_truth[i]))
    print(f"{image} and {gt} flipped")

    angle = random.choice(angles)
    cv2.imwrite(f"./rotated/raw/{image}_{angle}.jpg", generate_rotated_image(images[i],angle))
    cv2.imwrite(f"./rotated/warped/{gt}_{angle}.jpg", generate_rotated_image(ground_truth[i],angle))
    print(f"{image} and {gt} rotated in {angle} degrees")
    
    angle = random.choice(neg_angles)
    cv2.imwrite(f"./rotated/raw/{image}_{angle}.jpg", generate_rotated_image(images[i],angle))
    cv2.imwrite(f"./rotated/warped/{gt}_{angle}.jpg", generate_rotated_image(ground_truth[i],angle))
    print(f"{image} and {gt} rotated in {angle} degrees")

In [47]:
images_flipped = []
ground_truth_flipped = []

for image in glob.glob('./flipped/raw/*'):
    images_flipped.append(image)
    
for gt in glob.glob('./flipped/warped/*'):
    ground_truth_flipped.append(gt)

In [46]:
os.mkdir('./flipped/rotated/')
os.mkdir('./flipped/rotated/raw/')
os.mkdir('./flipped/rotated/warped/')

The code below rotate the flipped image ramdonly to a positive and negative angle

In [49]:
angles = [5,8,10]
neg_angles = [-5,-8,-10]

for i in range(len(images_flipped)):
    image = images_flipped[i].split("\\")[1].split('.')[0]
    gt = ground_truth_flipped[i].split("\\")[1].split('.')[0]

    angle = random.choice(angles)
    cv2.imwrite(f"flipped/rotated/raw/{image}_{angle}.jpg", generate_rotated_image(images_flipped[i],angle))
    cv2.imwrite(f"./flipped/rotated/warped/{gt}_{angle}.jpg", generate_rotated_image(ground_truth_flipped[i],angle))
    print(f"{image} and {gt} rotated in {angle} degrees")
    
    angle = random.choice(neg_angles)
    cv2.imwrite(f"flipped/rotated/raw/{image}_{angle}.jpg", generate_rotated_image(images_flipped[i],angle))
    cv2.imwrite(f"./flipped/rotated/warped/{gt}_{angle}.jpg", generate_rotated_image(ground_truth_flipped[i],angle))
    print(f"{image} and {gt} rotated in {angle} degrees")

021668_flipped and 021668_flipped rotated in 8 degrees
021668_flipped and 021668_flipped rotated in -5 degrees
021669_flipped and 021669_flipped rotated in 10 degrees
021669_flipped and 021669_flipped rotated in -5 degrees
021670_flipped and 021670_flipped rotated in 8 degrees
021670_flipped and 021670_flipped rotated in -10 degrees
021671_flipped and 021671_flipped rotated in 5 degrees
021671_flipped and 021671_flipped rotated in -8 degrees
021672_flipped and 021672_flipped rotated in 5 degrees
021672_flipped and 021672_flipped rotated in -8 degrees
021673_flipped and 021673_flipped rotated in 10 degrees
021673_flipped and 021673_flipped rotated in -10 degrees
021674_flipped and 021674_flipped rotated in 8 degrees
021674_flipped and 021674_flipped rotated in -8 degrees
021675_flipped and 021675_flipped rotated in 5 degrees
021675_flipped and 021675_flipped rotated in -10 degrees
021676_flipped and 021676_flipped rotated in 10 degrees
021676_flipped and 021676_flipped rotated in -10 de

021741_flipped and 021741_flipped rotated in -5 degrees
021742_flipped and 021742_flipped rotated in 10 degrees
021742_flipped and 021742_flipped rotated in -8 degrees
021743_flipped and 021743_flipped rotated in 10 degrees
021743_flipped and 021743_flipped rotated in -10 degrees
021744_flipped and 021744_flipped rotated in 10 degrees
021744_flipped and 021744_flipped rotated in -5 degrees
021745_flipped and 021745_flipped rotated in 10 degrees
021745_flipped and 021745_flipped rotated in -10 degrees
021746_flipped and 021746_flipped rotated in 10 degrees
021746_flipped and 021746_flipped rotated in -5 degrees
021747_flipped and 021747_flipped rotated in 8 degrees
021747_flipped and 021747_flipped rotated in -10 degrees
021748_flipped and 021748_flipped rotated in 8 degrees
021748_flipped and 021748_flipped rotated in -5 degrees
021749_flipped and 021749_flipped rotated in 5 degrees
021749_flipped and 021749_flipped rotated in -8 degrees
021750_flipped and 021750_flipped rotated in 8 d

021815_flipped and 021815_flipped rotated in 8 degrees
021815_flipped and 021815_flipped rotated in -8 degrees
021816_flipped and 021816_flipped rotated in 8 degrees
021816_flipped and 021816_flipped rotated in -5 degrees
021817_flipped and 021817_flipped rotated in 5 degrees
021817_flipped and 021817_flipped rotated in -10 degrees
021818_flipped and 021818_flipped rotated in 10 degrees
021818_flipped and 021818_flipped rotated in -5 degrees
021819_flipped and 021819_flipped rotated in 5 degrees
021819_flipped and 021819_flipped rotated in -10 degrees
021820_flipped and 021820_flipped rotated in 10 degrees
021820_flipped and 021820_flipped rotated in -5 degrees
021826_flipped and 021826_flipped rotated in 5 degrees
021826_flipped and 021826_flipped rotated in -8 degrees
021827_flipped and 021827_flipped rotated in 8 degrees
021827_flipped and 021827_flipped rotated in -5 degrees
021828_flipped and 021828_flipped rotated in 5 degrees
021828_flipped and 021828_flipped rotated in -5 degre

021893_flipped and 021893_flipped rotated in -10 degrees
021894_flipped and 021894_flipped rotated in 8 degrees
021894_flipped and 021894_flipped rotated in -8 degrees
021895_flipped and 021895_flipped rotated in 10 degrees
021895_flipped and 021895_flipped rotated in -10 degrees
021896_flipped and 021896_flipped rotated in 5 degrees
021896_flipped and 021896_flipped rotated in -5 degrees
021897_flipped and 021897_flipped rotated in 8 degrees
021897_flipped and 021897_flipped rotated in -8 degrees
021898_flipped and 021898_flipped rotated in 8 degrees
021898_flipped and 021898_flipped rotated in -5 degrees
021899_flipped and 021899_flipped rotated in 5 degrees
021899_flipped and 021899_flipped rotated in -8 degrees
021900_flipped and 021900_flipped rotated in 10 degrees
021900_flipped and 021900_flipped rotated in -5 degrees
021901_flipped and 021901_flipped rotated in 5 degrees
021901_flipped and 021901_flipped rotated in -10 degrees
021902_flipped and 021902_flipped rotated in 10 deg

021967_flipped and 021967_flipped rotated in 8 degrees
021967_flipped and 021967_flipped rotated in -10 degrees
021968_flipped and 021968_flipped rotated in 10 degrees
021968_flipped and 021968_flipped rotated in -5 degrees
021969_flipped and 021969_flipped rotated in 5 degrees
021969_flipped and 021969_flipped rotated in -8 degrees
021970_flipped and 021970_flipped rotated in 8 degrees
021970_flipped and 021970_flipped rotated in -5 degrees
021971_flipped and 021971_flipped rotated in 8 degrees
021971_flipped and 021971_flipped rotated in -8 degrees
021972_flipped and 021972_flipped rotated in 8 degrees
021972_flipped and 021972_flipped rotated in -8 degrees
021973_flipped and 021973_flipped rotated in 8 degrees
021973_flipped and 021973_flipped rotated in -10 degrees
021974_flipped and 021974_flipped rotated in 8 degrees
021974_flipped and 021974_flipped rotated in -8 degrees
021975_flipped and 021975_flipped rotated in 5 degrees
021975_flipped and 021975_flipped rotated in -5 degree

...

028091_flipped and 028091_flipped rotated in 8 degrees
028091_flipped and 028091_flipped rotated in -10 degrees
028092_flipped and 028092_flipped rotated in 5 degrees
028092_flipped and 028092_flipped rotated in -10 degrees
028093_flipped and 028093_flipped rotated in 10 degrees
028093_flipped and 028093_flipped rotated in -10 degrees
028094_flipped and 028094_flipped rotated in 5 degrees
028094_flipped and 028094_flipped rotated in -8 degrees
028095_flipped and 028095_flipped rotated in 5 degrees
028095_flipped and 028095_flipped rotated in -8 degrees
028096_flipped and 028096_flipped rotated in 8 degrees
028096_flipped and 028096_flipped rotated in -10 degrees
028097_flipped and 028097_flipped rotated in 5 degrees
028097_flipped and 028097_flipped rotated in -8 degrees
028098_flipped and 028098_flipped rotated in 5 degrees
028098_flipped and 028098_flipped rotated in -10 degrees
028099_flipped and 028099_flipped rotated in 5 degrees
028099_flipped and 028099_flipped rotated in -5 deg

028165_flipped and 028165_flipped rotated in 8 degrees
028165_flipped and 028165_flipped rotated in -5 degrees
028166_flipped and 028166_flipped rotated in 8 degrees
028166_flipped and 028166_flipped rotated in -5 degrees
028167_flipped and 028167_flipped rotated in 5 degrees
028167_flipped and 028167_flipped rotated in -8 degrees
028168_flipped and 028168_flipped rotated in 8 degrees
028168_flipped and 028168_flipped rotated in -5 degrees
028169_flipped and 028169_flipped rotated in 8 degrees
028169_flipped and 028169_flipped rotated in -10 degrees
028170_flipped and 028170_flipped rotated in 10 degrees
028170_flipped and 028170_flipped rotated in -5 degrees
028171_flipped and 028171_flipped rotated in 10 degrees
028171_flipped and 028171_flipped rotated in -10 degrees
028172_flipped and 028172_flipped rotated in 8 degrees
028172_flipped and 028172_flipped rotated in -10 degrees
028173_flipped and 028173_flipped rotated in 10 degrees
028173_flipped and 028173_flipped rotated in -8 deg

028239_flipped and 028239_flipped rotated in 8 degrees
028239_flipped and 028239_flipped rotated in -8 degrees
028240_flipped and 028240_flipped rotated in 8 degrees
028240_flipped and 028240_flipped rotated in -5 degrees
028241_flipped and 028241_flipped rotated in 5 degrees
028241_flipped and 028241_flipped rotated in -5 degrees
028242_flipped and 028242_flipped rotated in 10 degrees
028242_flipped and 028242_flipped rotated in -10 degrees
028243_flipped and 028243_flipped rotated in 5 degrees
028243_flipped and 028243_flipped rotated in -5 degrees
028244_flipped and 028244_flipped rotated in 5 degrees
028244_flipped and 028244_flipped rotated in -10 degrees
028245_flipped and 028245_flipped rotated in 5 degrees
028245_flipped and 028245_flipped rotated in -8 degrees
028246_flipped and 028246_flipped rotated in 8 degrees
028246_flipped and 028246_flipped rotated in -8 degrees
028247_flipped and 028247_flipped rotated in 10 degrees
028247_flipped and 028247_flipped rotated in -10 degr

028312_flipped and 028312_flipped rotated in -8 degrees
028313_flipped and 028313_flipped rotated in 8 degrees
028313_flipped and 028313_flipped rotated in -5 degrees
028314_flipped and 028314_flipped rotated in 5 degrees
028314_flipped and 028314_flipped rotated in -5 degrees
028315_flipped and 028315_flipped rotated in 8 degrees
028315_flipped and 028315_flipped rotated in -10 degrees
028316_flipped and 028316_flipped rotated in 8 degrees
028316_flipped and 028316_flipped rotated in -10 degrees
028317_flipped and 028317_flipped rotated in 5 degrees
028317_flipped and 028317_flipped rotated in -8 degrees
028318_flipped and 028318_flipped rotated in 10 degrees
028318_flipped and 028318_flipped rotated in -8 degrees
028319_flipped and 028319_flipped rotated in 8 degrees
028319_flipped and 028319_flipped rotated in -5 degrees
028320_flipped and 028320_flipped rotated in 8 degrees
028320_flipped and 028320_flipped rotated in -5 degrees
028321_flipped and 028321_flipped rotated in 10 degre

028386_flipped and 028386_flipped rotated in -5 degrees
028387_flipped and 028387_flipped rotated in 10 degrees
028387_flipped and 028387_flipped rotated in -10 degrees
028388_flipped and 028388_flipped rotated in 5 degrees
028388_flipped and 028388_flipped rotated in -8 degrees
028389_flipped and 028389_flipped rotated in 5 degrees
028389_flipped and 028389_flipped rotated in -5 degrees
028390_flipped and 028390_flipped rotated in 10 degrees
028390_flipped and 028390_flipped rotated in -10 degrees
028391_flipped and 028391_flipped rotated in 10 degrees
028391_flipped and 028391_flipped rotated in -5 degrees
028392_flipped and 028392_flipped rotated in 8 degrees
028392_flipped and 028392_flipped rotated in -8 degrees
028393_flipped and 028393_flipped rotated in 10 degrees
028393_flipped and 028393_flipped rotated in -8 degrees
028394_flipped and 028394_flipped rotated in 10 degrees
028394_flipped and 028394_flipped rotated in -5 degrees
028395_flipped and 028395_flipped rotated in 10 d